In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [35]:
df=pd.read_csv('/Users/vallirajasekar/Desktop/NLP_PROJECT/Fake_News/Fake_News_Classifier_LSTM/train 2.csv')

In [36]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [37]:
df.drop(['id'],axis=1,inplace=True)

In [38]:
df['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   author  18843 non-null  object
 2   text    20761 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 650.1+ KB


In [40]:
df.dropna(inplace=True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18285 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18285 non-null  object
 1   author  18285 non-null  object
 2   text    18285 non-null  object
 3   label   18285 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 714.3+ KB


In [42]:
df['label'].value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [43]:
df.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [44]:
df_1=df.sample(n=2500)

In [45]:
X=df_1.drop('label',axis=1)

In [46]:
y=df_1.label

In [47]:
X

,title,author,text
1946,Comment on Canadian Man Slams Americans Who Do...,kryptodyne,Subscribe \nA dumbfounded Canadian — Richard B...
19194,Latest WikiLeaks Vault 7 Release Claims CIA Ca...,Lucas Nolan,WikiLeaks’ latest release from the Vault 7 lea...
12850,"Ecuador Votes as Latin America’s Left Sags, a ...",Nicholas Casey and Maggy Ayala,"QUITO, Ecuador — For most of Luis García’s ..."
14270,"Ft. Lauderdale Shooter Booked, Then Canceled, ...",Frances Martel,"Esteban Santiago Ruiz, the gunman who killed f..."
13021,On ‘Patriots Day’: ‘Love Will Defeat Evil’ - T...,Cara Buckley,Here’s something Oscar voters and regular folk...
...,...,...,...
1733,Gainor: Left-Wing Media Smears Anti-Sharia March,Dan Riehl,Dan Gainor of the Media Research Center joined...
3434,Graft Fighter in Egypt Finds Himself a Defenda...,Declan Walsh,CAIRO — Egypt’s czar thought he was just ...
11893,US: 800-900 ISIS ‘Probably’ Killed in Mosul Fi...,Jason Ditz,Forces Still Several Kilometers From Mosul by ...
8676,Trump Talks Tough on U.S.-China Trade but Dela...,Keith Bradsher,SHANGHAI — President Trump appeared to foll...


In [48]:
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [49]:
## Vocabulary size
voc_size=5000

## One Hot Represenatation

In [50]:
message=X.copy()

In [55]:
message['title']

1946     Comment on Canadian Man Slams Americans Who Do...
19194    Latest WikiLeaks Vault 7 Release Claims CIA Ca...
12850    Ecuador Votes as Latin America’s Left Sags, a ...
14270    Ft. Lauderdale Shooter Booked, Then Canceled, ...
13021    On ‘Patriots Day’: ‘Love Will Defeat Evil’ - T...
                               ...                        
1733      Gainor: Left-Wing Media Smears Anti-Sharia March
3434     Graft Fighter in Egypt Finds Himself a Defenda...
11893    US: 800-900 ISIS ‘Probably’ Killed in Mosul Fi...
8676     Trump Talks Tough on U.S.-China Trade but Dela...
10499                                   The Loosening Grip
Name: title, Length: 2500, dtype: object

In [57]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [58]:
ps=PorterStemmer()

In [61]:
message.reset_index(drop=True, inplace=True)


In [62]:
corpus=[]

for i in range(0,len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [63]:
corpus

['comment canadian man slam american realiz awesom obama kryptodyn',
 'latest wikileak vault releas claim cia attribut cyber attack countri breitbart',
 'ecuador vote latin america left sag gener icon gone new york time',
 'ft lauderdal shooter book cancel new year eve flight new york',
 'patriot day love defeat evil new york time',
 'caitlyn jenner look run offic bring gop around lgbt issu breitbart',
 'geraldo rip yale student symbol hunger strike everyth wrong millenni activ activ without sacrific breitbart',
 'fmr fnc host alisyn camerota come sexual harass scandal breitbart',
 'wizard come stage harri potter curs child begin preview london new york time',
 'press secretari affirm trump believ lie million illeg voter new york time',
 'watch world seri big news broke',
 'georgia man sentenc life without parol son death hot car new york time',
 'trump warn vote flip machin',
 'sander trump tell truth campaign term would presid breitbart',
 'hillari clinton hit unfavor high higher tru

## One Hot Representation

In [64]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [65]:
onehot_repr

[[3708, 4938, 942, 3665, 1292, 3107, 806, 629, 265],
 [2341, 496, 1289, 1390, 2617, 839, 3591, 1823, 2070, 4778, 1328],
 [1730, 1245, 4549, 2569, 161, 2379, 4241, 3424, 1082, 3412, 1109, 3358],
 [2379, 2954, 2635, 4432, 482, 3412, 4818, 4156, 3682, 3412, 1109],
 [4356, 4566, 1998, 4873, 4154, 3412, 1109, 3358],
 [2338, 215, 1752, 1680, 510, 4885, 2141, 4102, 2711, 2846, 1328],
 [1928,
  1188,
  982,
  3671,
  4538,
  430,
  3791,
  4632,
  1024,
  1644,
  1064,
  1064,
  1869,
  2973,
  1328],
 [2202, 376, 2439, 4190, 3258, 492, 1516, 52, 1102, 1328],
 [2266, 492, 3600, 4340, 2896, 4380, 2638, 1246, 1261, 2232, 3412, 1109, 3358],
 [501, 1823, 2082, 2901, 327, 3963, 2685, 1493, 2526, 3412, 1109, 3358],
 [1116, 3842, 3850, 2912, 1245, 309],
 [3293, 942, 3555, 1140, 1869, 3895, 3144, 4062, 3228, 3354, 3412, 1109, 3358],
 [2901, 3675, 1245, 780, 3090],
 [4462, 2901, 2935, 4708, 2628, 4649, 132, 4969, 1328],
 [872, 3494, 417, 4946, 4908, 2532, 2901],
 [3938, 4460, 1549, 4152, 2890, 2070, 81

In [66]:
corpus[1]

'latest wikileak vault releas claim cia attribut cyber attack countri breitbart'

In [67]:
onehot_repr[1]

[2341, 496, 1289, 1390, 2617, 839, 3591, 1823, 2070, 4778, 1328]

## Embedding Representation

In [68]:
sent_length=20

In [69]:
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [70]:
embedded_docs

array([[   0,    0,    0, ...,  806,  629,  265],
       [   0,    0,    0, ..., 2070, 4778, 1328],
       [   0,    0,    0, ..., 3412, 1109, 3358],
       ...,
       [   0,    0,    0, ..., 1549, 2975, 3444],
       [   0,    0,    0, ..., 3412, 1109, 3358],
       [   0,    0,    0, ...,    0, 4696, 4583]], dtype=int32)

## Creating the LSTM Model

In [72]:
embedding_vector_features=40 #Embedded docs is converted into 40 vectors
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [74]:
len(embedded_docs),y.shape

(2500, (2500,))

In [75]:
import numpy as np

In [79]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [80]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2)

In [82]:
## Finally Training the Model

model.fit(X_final,y_final,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
40/40 [==============================] - 1s 15ms/step - loss: 0.6138 - accuracy: 0.6812 - val_loss: 0.5289 - val_accuracy: 0.6400
Epoch 2/10
40/40 [==============================] - 0s 11ms/step - loss: 0.3309 - accuracy: 0.8456 - val_loss: 0.1942 - val_accuracy: 0.9560
Epoch 3/10
40/40 [==============================] - 0s 12ms/step - loss: 0.1534 - accuracy: 0.9428 - val_loss: 0.0773 - val_accuracy: 0.9800
Epoch 4/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0736 - accuracy: 0.9796 - val_loss: 0.0324 - val_accuracy: 0.9920
Epoch 5/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0428 - accuracy: 0.9904 - val_loss: 0.0162 - val_accuracy: 0.9960
Epoch 6/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0215 - accuracy: 0.9956 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 7/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0141 - accuracy: 0.9964 - val_loss: 0.0073 - val_accuracy: 1.0000
Epoch 8/10
40

## Adding Dropout 

In [83]:
y_pred=model.predict(X_test)

16/16 [==============================] - 0s 2ms/step


In [95]:
y_pred= (y_pred > 0.5).astype(float)

In [96]:
from sklearn.metrics import confusion_matrix

In [97]:
confusion_matrix(y_test,y_pred)

array([[271,   0],
       [  0, 229]])

In [98]:
from sklearn.metrics import accuracy_score

In [99]:
accuracy_score(y_test,y_pred)

1.0

In [100]:
from sklearn.metrics import classification_report

In [102]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       271
           1       1.00      1.00      1.00       229

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500

